# Networking Basics

TCP(Transmission Control Protocol) is a **connection-oriented** communication protocol that provides a reliable flow of data between two computers.

Analogy: Speaking on phone

Example applications:

* HTTP, FTP, Telnet
* Skype uses TCP for call signaling, and both UDP and TCP for transporting media traffic

UDP(User Datagram Protocol) is a connectionless communication protocol that sends independent packets of data, called datagrams, from one computer to another with no guarantees about arrival or order of arrival. Every datagram contains all the details about who I am, where should it go.

Similar to sending multiple emails/letters to friends, each containing part of a message.

Example:

* Clock server
* Ping
* Live streaming(event/sports broadcasting)

The TCP and UDP protocols use ports to map incoming data to a particular process running on a computer.

* Port is represented by a positive(16-bit) integer value
* Some ports have been reserved to support common/well known services:
    * ftp 21/tcp
    * telnet 23/tcp
    * smtp 25/tcp
    * login 513/tcp
* User-level processes/services generally use port number value >= 1024

# Sockets

* Sockets provide an interface for programming networks at the transport layer
* Network communication using Sockets is very much similar to performing file I/O
    * In fact, socket handle is treated like file handles
    * The streams used in file I/O operation are also applicable to socket-based I/O
* Socket-based communication is programming language independent. - That means, a socket program written in Java language can also communicate to a program written in Java or non-Java socket program

A server(program) runs on a specific computer and has a socket that is bound to a specific port. The server waits and listens to the socket for a client to make a connection request.

<img src="img/img02.png" width="400">

If everything goes well, the server accepts the connection. Upon acceptance, the server gets a new socket bounds to a different port. It needs a new socket(consequently a different port number) so that it can continue to listen to the original socket for connection requests while serving the connected client.

<img src="img/img03.png" width="400">

* A socket is an endpoint of a two-way communication link between two programs running on the network
* A socket is bound to a port number so that the TCP layer can identify the application that data destined to be sent
* Java's `.net` package provides two classes:
    * Socket - for implementing a client
    * ServerSocket - for implementing a server
    
# Implementing a Server

1. Open the Server Socket
```
ServerSocket server;
DataOutputStream os;
DataInputStream is;
server = new ServerSocket(PORT);
```
2. Wait for the client request
```
Socket client = server.accept();
```
3. Create I/O streams for communicating to the client
```
is = new DataInputStream(client.getInputStream());
os = new DataOutputStream(client.getOutputStream());
```
4. Perform communication with client
```
Receive from client: String line = is.readLine();
Send to client: os.writeBytes("Hello\n");
```
5. Close sockets
```
client.close();
```

For multi-threaded server:
```
while(true) {
    wait for client request(step 2 above)
    create a thread with "client" socket as parameter(the thread creates streams(step 3) and does communications Remove thread once service is provided)
}
```

# Implementing a Client

1. Create a Socket Object
```
client = new Socket(server, port_id);
```
2. Create I/O streams for communicating with the server.
```
is = new DataInputStream(client.getInputStream());
os = new DataOutputStream(client.getOutputStream());
```
3. Perform I/O or communication with the server
```
String line = is.readLine();
os.writeBytes("Hello\n");
```
4. Close the socket when done
```
client.close();
```

# Simplified Code

A simple server:

```
import java.net.*;
import java.io.*;
public class SimpleServer {
    public static void main(String args[]) throws IOException {
        //Register service on port 1234
        ServerSocket s = new ServerSocket(1234);
        Socket s1 = s.accept();
        OutputStream s1out = s1.getOutputStream();
        DataOutputStream dos = new DataOutputStream(s1out);
        dos.writeUTF("Hi there");
        //close the connection, but not the server socket
        dos.close();
        s1out.close();
        s1.close();
    }
}
```

A simple client:

```
import java.net.*;
import java.io.*;
public class SimpleClient {
    public static void main(String args[]) throws IOException {
        Socket s1 = new Socket("jarrett.cis.unimelb.edu.au", 1234);
        InputStream s1In = s1.getInputStream();
        DataInputStream dis = new DataInputStream(s1In);
        String st = new String(dis.readUTF());
        System.out.println(st);
        dis.close();
        s1In.close();
        s1.close();
}
```

A server in loop:

```
//a simple server program that runs forever in a single thread
import java.net.*;
import java.io.*;
public class SimpleServerLoop {
    public static void main(String args[]) throws IOException {
        ServerSocket s = new ServerSocket(1234);
        while(true) {
            Socket s1 = s.accept();
            OutputStream s1out = s1.getOutputStream();
            DataOutputStream dos = new DataOutputStream(s1out);
            dos.writeUTF("Hi there");
            dos.close();
            s1out.close();
            s1.close();
        }
    }
}
```

# Exception

* public `ServerSocket(int port)` throws IOException
    * Creates a server socket on a specified port
    * A port of $0$ creates a socket on any free port. You can use `getLocalPort()` to identify the assigned port on which this socket is listening
    * The maximum queue length for incoming connection indications(a request to connect) is set to $50$. If a connection indication arrives when the queue is full, the connection is refused.
* Throws:
    * IOException - if an I/O error occurs when opening the socket
    * SecurityException - if a security manager exists and its checkListen method doesn't allow the operation

# Java API for UDP Programming

* Java API provides datagram communication by means of two classes
    * DatagramPacket `|Msg|length|Host|serverPort|`
    * DatagramSocket
    
## UDP Client

```
import java.net.*;
import java.io.*;
public class UDPClient {
    public static void main(String args[]) {
        //args give message contents and server hostname
        //"Usage: java UDPClient <message> <Host name> <Port number>"
        DatagramSocket aSocket = null;
        try {
            aSocket = new DatagramSocket();
            byte[] m = args[0].getBytes();
            InetAddress aHost = InetAddress.getByName(args[1]);
            int serverPort = 6789; // Or Integer.valueOf(args[2]).intValue() if use <Port number>
            DatagramPacket request = new DatagramPacket(m, args[0].length(), aHost, serverPort);
            aSocket.send(request);
            byte[] buffer = new byte[1000];
            DatagramPacket reply = new DatagramPacket(buffer, buffer.length);
            aSocket.receive(reply);
            System.out.println("Reply: " + new String(reply.getData()));
        }
        catch (SocketException e) {System.out.println("Socket: " + e.getMessage());}
        catch (IOException e) {System.out.println("IO " + e.getMessage());}
        finally {
            if(aSocket != null)
                aSocket.close();
        }
    }
}
```

## UDP Server

```
import java.net.*;
import java.io.*;
public class UDPServer {
    public static void main(String args[]) {
        DatagramSocket aSocket = null;
        try {
            aSocket = new DatagramSocket(6789); //fixed port number
            byte[] buffer = new byte[1000];
            while(true) {
                DatagramPacket request = new DatagramPacket(buffer, buffer.length);
                aSocket.receive(request);
                DatagramPacket reply = new DatagramPacket(request.getData(), request.getLength(),
                    request.getAddress(), request.getPort());
                aSocket.send(reply);
            }
        }
        catch (SocketException e) {System.out.println("Socket: " + e.getMessage());}
        catch (IOException e) {System.out.println("IO " + e.getMessage());}
        finally {
            if(aSocket != null)
                aSocket.close();
        }
    }
}
```